# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [53]:
import graphlab
import math
import random
import numpy
from matplotlib import pyplot as plt

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [54]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [55]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [56]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [57]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.207423     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.222383     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [58]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [141]:
print_coefficients(model_all)
print all_features[13]
print all_features[12]
print all_features[4]
print all_features[3]
print all_features[2]


Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05
sqft_above
grade
sqft_living_sqrt
sqft_living
bathrooms


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [150]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [151]:
l1_penalty_g=numpy.logspace(1, 7, num=13)

In [152]:
l1_penalty_g

array([  1.00000000e+01,   3.16227766e+01,   1.00000000e+02,
         3.16227766e+02,   1.00000000e+03,   3.16227766e+03,
         1.00000000e+04,   3.16227766e+04,   1.00000000e+05,
         3.16227766e+05,   1.00000000e+06,   3.16227766e+06,
         1.00000000e+07])

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [153]:
i=0
iteration=[]
ltemp_l=[]
rss_l=[]
for ltemp in l1_penalty_g:
    model= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=ltemp)
    prediction = model.predict(validation)
    rss=((prediction-validation['price'])**2).sum()
    line= "Iteration",i,"   ","ltemp",ltemp,"   ","RSS",rss
    print line
    print_coefficients(model)
    iteration.append(i)
    ltemp_l.append(ltemp)
    rss_l.append(rss)
    i+=1



    
    

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.160574     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

('Iteration', 12, '   ', 'ltemp', 10000000.0, '   ', 'RSS', 625883719085425.0)
Learned polynomial for degree 17:
       17         16       15         14        13        12
55.33 x  + 9.395 x  + 122 x  + 43.26 x  + 6200 x  + 6585 x 
              11             10             9             8         7
 + 9.309e+04 x  + 5.985e+05 x  + 1.287e+04 x + 2.115e+04 x + 147.9 x
             6        5         4             3         2
 - 0.000824 x + 1124 x + 39.11 x + 2.539e+04 x + 928.1 x + 7923 x + 1.96e+04


In [154]:
print rss_l

[625766285142460.5, 625766285362394.4, 625766286057885.1, 625766288257224.4, 625766295212186.1, 625766317206080.8, 625766386760658.0, 625766606749278.5, 625767302791634.9, 625769507643885.8, 625776517727024.5, 625799062845466.6, 625883719085425.0]


In [155]:
c1=graphlab.SArray(iteration)
c2=graphlab.SArray(ltemp_l)
c3=graphlab.SArray(rss_l)

In [156]:
results = graphlab.SFrame({'i':c1,'l1':c2,'rss':c3})
results


i,l1,rss
0,10.0,6.25766285142e+14
1,31.6227766017,6.25766285362e+14
2,100.0,6.25766286058e+14
3,316.227766017,6.25766288257e+14
4,1000.0,6.25766295212e+14
5,3162.27766017,6.25766317206e+14
6,10000.0,6.25766386761e+14
7,31622.7766017,6.25766606749e+14
8,100000.0,6.25767302792e+14
9,316227.766017,6.25769507644e+14


In [157]:
results.sort('rss')

i,l1,rss
0,10.0,6.25766285142e+14
1,31.6227766017,6.25766285362e+14
2,100.0,6.25766286058e+14
3,316.227766017,6.25766288257e+14
4,1000.0,6.25766295212e+14
5,3162.27766017,6.25766317206e+14
6,10000.0,6.25766386761e+14
7,31622.7766017,6.25766606749e+14
8,100000.0,6.25767302792e+14
9,316227.766017,6.25769507644e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [159]:
model12= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10)
print_coefficients(model12)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.169520     | 5655864.782013     | 285354.826809 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

Learned polynomial for degree 17:
       17         16         15         14        13        12
56.07 x  + 9.434 x  + 122.4 x  + 43.29 x  + 6207 x  + 6609 x 
              11             10             9            8         7
 + 9.331e+04 x  + 6.019e+05 x  + 1.292e+04 x + 2.12e+04 x + 148.3 x
             6        5         4             3       2
 + 0.003484 x + 1125 x + 39.12 x + 2.541e+04 x + 937 x + 7937 x + 1.899e+04


This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [91]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [130]:
l1_penalty_values = numpy.logspace(8, 10, num=20)
len(l1_penalty_values)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


In [104]:
i2=0
iteration2=[]
ltemp_l2=[]
rss_l2=[]
m_nonzero=[]

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [131]:

for ltemp2 in l1_penalty_values:
    model= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=ltemp2)
    m_count=-1
    for i in model.coefficients['value']:
        if i!=0:
            m_count+=1
    m_nonzero.append(m_count)
    print_coefficients(model)
    prediction = model.predict(testing)
    
    rss2=((prediction-testing['price'])**2).sum()
    line= "Iteration",i2,"   ","ltemp",ltemp2,"   ","RSS",rss2
    print line
 
    iteration2.append(i2)
    ltemp_l2.append(ltemp2)
    rss_l2.append(rss2)
    i2+=1


Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000012  | 0.151596     | 7187431.218291     | 380705.223778 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Learned polynomial for degree 17:
 
5.645e+05
('Iteration', 39, '   ', 'ltemp', 10000000000.0, '   ', 'RSS', 286162037256993.1)


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [106]:
print rss_l2

[156998193279209.3, 157061573462156.8, 157147654144329.16, 157318167494408.25, 157639541100053.38, 158126272293670.94, 158905814085727.8, 160202272804961.44, 162325904156613.06, 165400999406042.2, 169935261645333.03, 176962594541356.5, 188644538886643.03, 204434504732854.12, 225420664589747.34, 251258967249544.8, 289400419739480.4, 322281825795624.1, 286765188562251.25, 286162037256993.1]


In [119]:
c21=graphlab.SArray(iteration2)
c22=graphlab.SArray(ltemp_l2)
c23=graphlab.SArray(rss_l2)
c24=graphlab.SArray(m_nonzero)

results2 = graphlab.SFrame({'i':c21,'l1':c22,'rss':c23,'nonzero':c24})
results2.print_rows(20)


+----+---------------+---------+-------------------+
| i  |       l1      | nonzero |        rss        |
+----+---------------+---------+-------------------+
| 0  |  100000000.0  |    18   | 1.56998193279e+14 |
| 1  |  127427498.57 |    18   | 1.57061573462e+14 |
| 2  | 162377673.919 |    18   | 1.57147654144e+14 |
| 3  | 206913808.111 |    18   | 1.57318167494e+14 |
| 4  | 263665089.873 |    17   |  1.576395411e+14  |
| 5  | 335981828.628 |    17   | 1.58126272294e+14 |
| 6  | 428133239.872 |    17   | 1.58905814086e+14 |
| 7  | 545559478.117 |    17   | 1.60202272805e+14 |
| 8  | 695192796.178 |    17   | 1.62325904157e+14 |
| 9  |  885866790.41 |    16   | 1.65400999406e+14 |
| 10 | 1128837891.68 |    15   | 1.69935261645e+14 |
| 11 | 1438449888.29 |    15   | 1.76962594541e+14 |
| 12 | 1832980710.83 |    13   | 1.88644538887e+14 |
| 13 | 2335721469.09 |    12   | 2.04434504733e+14 |
| 14 | 2976351441.63 |    10   |  2.2542066459e+14 |
| 15 | 3792690190.73 |    6    |  2.5125896725

In [132]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [133]:
l1_penalty_values_3 = numpy.linspace(l1_penalty_min,l1_penalty_max,20)
l1_penalty_values_3

array([  2.97635144e+09,   3.01931664e+09,   3.06228184e+09,
         3.10524703e+09,   3.14821223e+09,   3.19117743e+09,
         3.23414263e+09,   3.27710782e+09,   3.32007302e+09,
         3.36303822e+09,   3.40600341e+09,   3.44896861e+09,
         3.49193381e+09,   3.53489901e+09,   3.57786420e+09,
         3.62082940e+09,   3.66379460e+09,   3.70675980e+09,
         3.74972499e+09,   3.79269019e+09])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [137]:
message=[]
count_l=[]

for ltemp in l1_penalty_values_3:
    model= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=ltemp)
    m_count=0
    for i in model.coefficients['value']:
        if i!=0:
            m_count+=1
    count_l.append(m_count)
    print_coefficients(model)
    if m_count==8:
        prediction = model.predict(validation)
        myrss=((prediction-validation['price'])**2).sum()
        note= 'rss is', myrss, 'at l1 ',ltemp
        print note
        model_fit.append(print_coefficients(model))
        message.append(note)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.161571     | 6697293.370032     | 370685.051067 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Learned polynomial for degree 17:
       14        13         5         4             3
27.49 x  + 2265 x  + 592.2 x + 30.56 x + 1.384e+04 x + 2.403e+05


In [138]:
print count_l
print message


[10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6]
[('rss is', 1034616909232827.4, 'at l1 ', 3320073020.198421), ('rss is', 1038554735941040.2, 'at l1 ', 3363038217.5194736), ('rss is', 1043237237871701.2, 'at l1 ', 3406003414.8405266)]
[None, None, None]


In [139]:
model_best= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3320073020.198421)
print_coefficients(model_best)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.166526     | 6581468.832455     | 356349.534440 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

Learned polynomial for degree 17:
       14        13         8         5         4            3
30.82 x  + 3100 x  + 829.6 x + 719.9 x + 32.94 x + 1.65e+04 x + 1108 x + 2.152e+05


This model may not be optimal. To improve it, consider increasing `max_iterations`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [142]:
print all_features[13]
print all_features[12]
print all_features[7]
print all_features[4]
print all_features[3]
print all_features[2]
print all_features[0]


sqft_above
grade
floors
sqft_living_sqrt
sqft_living
bathrooms
bedrooms
